## 8.6 Quantum Key Distribution using StrangelyQuantum
This example demonstrates the BB84 algorithm for QKD.

In [ ]:
using ImageShow
using Random
using StrangelyDisplayed
using StrangelyQuantum

In this algorithm, the sender chooses a random key (in this example, 8 bits long) and a random set of base bits with the same length. The receiver also chooses a random set of base bits (again with the same length).

The sender sends the qubits, setting them to 0 or 1 according to the key. The sender applies a Hadamard gate to qubits if the corresponding base bit is 1.

The receiver applies a Hadamard gate to received bits for which the receiver's corresponding base bits are 1. The message has be sent and received, but not yet decrypted.

The sender can now inform the receiver (even using an open channel) of the sender's base bits. The receiver knows now that if a sender's base bit matches the corresponding receiver's base bit, that the measured value of the message bit is valid.

In [ ]:
function bb84()
    # Alice and Bob will exchange SIZE qubits, hence the resulting
    # key will be maximum SIZE bits.
    SIZE = 8
    random = Xoshiro()

    aliceBits = Vector{Bool}(undef, SIZE)  # random bits chosen by Alice
    bobBits = Vector{Bool}(undef, SIZE)  # bits measured by Bob
    aliceBase = Vector{Bool}(undef, SIZE) # random bases chosen by Alice
    bobBase = Vector{Bool}(undef, SIZE) # random bases chosen by Bob

    simulator = SimpleQuantumExecutionEnvironment()
    program = Program(SIZE)
    prepareStep = Step()
    superPositionStep = Step()
    superPositionStep2 = Step()
    measureStep = Step()

    for i = 1:SIZE
        aliceBits[i] = rand(random, Bool)
        # if Alice's bit is 1, apply a X gate to the |0> state
        aliceBits[i] && addGate(prepareStep, X(i))
        aliceBase[i] = rand(random, Bool)
        # if Alice's base for this bit is 1, apply a H gate
        aliceBase[i] && addGate(superPositionStep, Hadamard(i))
        bobBase[i] = rand(random, Bool)
        # if Bob decides to measure in base 1, apply a H gate
        bobBase[i] && addGate(superPositionStep2, Hadamard(i))
        # Finally, Bob measures the result
        addGate(measureStep, Measurement(i))
    end

    addStep(program, prepareStep)
    addStep(program, superPositionStep)
    addStep(program, superPositionStep2)
    addStep(program, measureStep)

    result = runProgram(simulator, program)
    qubit = getQubits(result)

    measurement = Vector{Int}(undef, SIZE)
    key = IOBuffer()
    for i = 1:SIZE
        measurement[i] = measure(qubit[i])
        bobBits[i] = measurement[i] == 1
        if aliceBase[i] != bobBase[i]
            # If the random bases chosen by Alice and Bob for this bit
            # are different, ignore values
            println("Different bases used, ignore values.")
        else
            # Alice and Bob picked the same bases. The inital value from
            # Alice matches the measurement from Bob.
            # this bit now becomes part of the secret key
            println(
                "Same bases used. Alice sent ",
                aliceBits[i] ? '1' : '0',
                " and Bob received ",
                bobBits[i] ? '1' : '0',
            )
            print(key, aliceBits[i] ? "1" : "0")
        end
    end
    println("Secret key = ", String(take!(key)))

    return program
end

In [ ]:
program = bb84()

In [ ]:
drawProgram(program)

As in the [previous example](ch08-04-guess.ipynb) the correct values are measured if either zero or two Hadamard gates are applied. In this case communication between sender and receiver can establish which values are the correct ones, without disclosing the values.